In [16]:
import os
import pandas as pd
import openpyxl
from fuzzywuzzy import fuzz
import requests
import pandas as pd
from bs4 import BeautifulSoup
#import credentials
import re

In [40]:
def concat(archivo1, archivo2, archivo_resultado):
    # Cargar los dos archivos de Excel
    df1 = pd.read_excel(archivo1)
    df2 = pd.read_excel(archivo2)

    # Convertir la columna 'Código de barras' a tipo string en ambos DataFrames
    df1['Código de barras'] = df1['Código de barras'].astype(str)
    df2['Código de barras'] = df2['Código de barras'].astype(str)

    # Crear un DataFrame vacío para almacenar los resultados
    df_result = pd.DataFrame(columns=['Identificador de URL', 'canal_Nombre', 'Precio', 'canal_SKU', 'canal_Código de barras','canal_Costo',
                                      'df2_SKU', 'df2_Nombre', 'df2_Código de barras', 'df2_Costo', 'similarity', 'Marca'])

    # Iterar sobre las filas del primer DataFrame (df1)
    for i, row1 in df1.iterrows():
        canal_Nombre = row1['Nombre']
        Precio = row1['Precio']
        canal_SKU = row1['SKU']
        canal_Codigo = row1['Código de barras']
        canal_Identificador = row1['Identificador de URL']
        canal_Costo = row1['Costo']
        found = False

        # Iterar sobre las filas del segundo DataFrame (df2)
        for j, row2 in df2.iterrows():
            df2_Nombre = row2['Nombre']
            Costo = row2['Costo']
            df2_SKU = row2['SKU']
            df2_Codigo = row2['Código de barras']

            # Calcular la similitud entre los SKUs de df1 y df2 usando FuzzyWuzzy
            similarity = fuzz.token_set_ratio(canal_Codigo, df2_Codigo)

            # Si la similitud es mayor o igual al 99%, agregar la fila al DataFrame de resultados
            if similarity >= 99:
                df_result.loc[len(df_result)] = [canal_Identificador, canal_Nombre, Precio, canal_SKU, canal_Codigo, canal_Costo,
                                                  df2_SKU, df2_Nombre, df2_Codigo, Costo, similarity, row2.get("Marca", "")]
                found = True

        # Si no se encontró una coincidencia, agregar una fila con los datos de df1 y campos vacíos para df2
        if not found:
            df_result.loc[len(df_result)] = [canal_Identificador, canal_Nombre, Precio, canal_SKU, canal_Codigo, canal_Costo,
                                             None, None, None, None, None, None]

    # Iterar sobre las filas del segundo DataFrame (df2) para agregar filas no coincidentes al DataFrame de resultados
    for i, row2 in df2.iterrows():
        df2_Nombre = row2['Nombre']
        Costo = row2['Costo']
        df2_SKU = row2['SKU']
        df2_Codigo = row2['Código de barras']
        found = False

        # Verificar si el SKU de df2 ya está en el DataFrame de resultados
        for j, row_result in df_result.iterrows():
            if row_result['df2_SKU'] == df2_SKU:
                found = True
                break

        # Si no se encontró una coincidencia, agregar una fila con los datos de df2 y campos vacíos para df1
        if not found:
            df_result.loc[len(df_result)] = [None, None, None, None, None, None, 
                                             df2_SKU, df2_Nombre, df2_Codigo, Costo, None, row2.get("Marca", "")]

    df_result = df_result[['Identificador de URL', 'canal_Nombre', 'Precio', 'Marca', 'canal_SKU', 'canal_Código de barras', 'canal_Costo',
                           'df2_SKU', 'df2_Nombre', 'df2_Código de barras', 'df2_Costo', 'similarity']]
    

    # Reemplazar las comas por un espacio vacío en las columnas 'canal_Costo' y 'df2_Costo'
    df_result['canal_Costo'] = df_result['canal_Costo'].astype(str).str.replace(',', '')
    df_result['df2_Costo'] = df_result['df2_Costo'].astype(str).str.replace(',', '')


    df_result['canal_Costo'] = pd.to_numeric(df_result['canal_Costo'], errors='coerce')
    df_result['df2_Costo'] = pd.to_numeric(df_result['df2_Costo'], errors='coerce')


    df_result['Porcentaje de aumento'] = ((df_result['df2_Costo'] - df_result['canal_Costo']) / df_result['canal_Costo']) * 100


    total_rows = len(df_result)
    exact_match_count = len(df_result[df_result['similarity'] == 100])
    missing_df1_rows = len(df_result[df_result['df2_SKU'].isnull()])
    missing_df2_rows = len(df_result[df_result['canal_SKU'].isnull()])


    # Filtrar el DataFrame para incluir sólo los productos con un aumento mayor a 0%
    increased_products = df_result[df_result['Porcentaje de aumento'] > 0]

    # Calcular el aumento promedio sólo con los productos filtrados
    average_increase = increased_products['Porcentaje de aumento'].mean()

    print(f"En general, los productos aumentaron en un {average_increase:.2f}%.")

    excessive_threshold = 10
    excessive_increases = df_result[df_result['Porcentaje de aumento'] > excessive_threshold]

    if len(excessive_increases) > 0:
        print(f"Se encontraron {len(excessive_increases)} productos con un aumento excesivo:")
        for idx, row in excessive_increases.iterrows():
            print(f"  - {row['canal_Nombre']} (SKU: {row['canal_SKU']}) aumentó un {row['Porcentaje de aumento']:.2f}%.")
    else:
        print("No se encontraron aumentos excesivos en los productos.")
    

    print()
    print(f"Total de filas en df_result: {total_rows}")
    print(f"Filas con similaridad del 100%: {exact_match_count}")
    print(f"Filas de CANAL no encontradas en df2: {missing_df1_rows}")
    print(f"Filas de df2 no encontradas en CANAL: {missing_df2_rows}")
    print()
    df_result.to_excel(archivo_resultado, index=False)
    print("Archivo CONCATENADO guardado como",archivo_resultado)


In [4]:
# FUREY
concat('./procesadosCanal/furey.xlsx','./procesados/furey.xlsx','./listos/furey.xlsx')

Archivo CONCATENADO guardado como ./listos/furey.xlsx


In [41]:
# DRIMEL
concat('./procesadosCanal/drimel.xlsx','./procesados/drimel.xlsx','./listos/drimel.xlsx')

En general, los productos aumentaron en un 6.27%.
Se encontraron 16 productos con un aumento excesivo:
  - BabySec - Pañal Premium Hiper - Mediano - 48 Unidades (SKU: Drimel908) aumentó un 11.66%.
  - BabySec - Pañal Premium Jumbo - XGrande - 48 Unidades (SKU: Drimel950) aumentó un 17.15%.
  - BabySec - Pañal Premium Jumbo - XXGrande - 44 Unidades (SKU: Drimel951) aumentó un 17.15%.
  - Babysec - Pañal Ultra Regular - XXGrande - 8 Unidades (SKU: Drimel1393) aumentó un 12.43%.
  - Babysec - Pañal Premium Hiper - Grande - 40 Unidades (SKU: Drimel1572) aumentó un 11.66%.
  - Babysec - Pañal Premium Hiper - XGrande - 32 Unidades (SKU: Drimel1573) aumentó un 11.66%.
  - Babysec - Pañal Ultra Hiper - Vrom - XGrande - 28 Unidades (SKU: Drimel3533) aumentó un 11.82%.
  - Babysec - Pañal Ultra Hiper - Vrom - Grande - 36 Unidades (SKU: Drimel5695) aumentó un 11.82%.
  - Babysec - Pañal Ultra Hiper - Vrom - Mediano - 44 Unidades (SKU: Drimel6789) aumentó un 11.82%.
  - Babysec - Pañal Ultra Regul